# Carregando bibliotecas

In [2]:
# Importa todas as bibliotecas necessárias
import numpy as np
import pandas as pd
from datetime import datetime
import csv
import unidecode

# Dicionário de dados

**strRazaoSocial:** Razão social do fornecedor (empresa) na base de dados do Sindec

strNomeFantasia: Nome fantasia do fornecedor na base dedados do Sindec (nome comercial / popular / fachada)

**RadicalCNPJ:** Aplica-se para pessoa jurídica e serve para agrupar as informações de um mesmo fornecedor (matriz e filiais), sendo os oitos primeiros dígitos do número do CNPJ - Exemplo: a matriz (central) do banco e suas filiais (agências)

**RazaoSocialRFB:** Razão social do fornecedor na base de dados da RFB – Receita Federal do Brasil.

NomeFantasiaRFB: Nome fantasia do fornecedor na base de dados da RFB – Receita Federal do Brasil
Obs.: somente para os CNPJs (NumeroCNPJ) válidos na base da RFB

**CNAEPrincipal:** Código identificador da Classificação Nacional de Atividades Econômicas principal do fornecedor.
Obs.: somente para os CNPJs (NumeroCNPJ) válidos na base da RFB

DescCNAEPrincipal: Descrição da Classificação Nacional de Atividades Econômicas principal dofornecedor. 
Obs.: somente para os CNPJs (NumeroCNPJ) válidos na base da RFB

**Atendida:** Código identificador da reclamação fundamentada atendida ou não pela empresa/fornecedor:
S – Atendida
N – NÃO Atendida



# Verificando quantidade de dados no arquivo de 2014

In [5]:
df2014 = pd.read_csv('reclamacoes-fundamentadas-sindec-2014.csv')
df2014['AnoCalendario'].count()

267764

# Removendo colunas não utilizadas

In [6]:
df2014.drop(['DataAbertura', 'DataArquivamento', 'FaixaEtariaConsumidor', 'Atendida', 'DescricaoProblema', 'AnoCalendario', 'CodigoRegiao', 'CodigoAssunto', 'Tipo', 'CodigoProblema', 'RazaoSocialRFB', 'NomeFantasiaRFB','SexoConsumidor', 'CNAEPrincipal', 'RadicalCNPJ', 'DescCNAEPrincipal', 'CEPConsumidor'], axis=1, inplace=True)

# Removendo acentuações
No banco de dados, existe ocorrências de uma mesma razão social com e sem acentuação.

In [10]:
def remove_acentuacao(a):
    return unidecode.unidecode(a.decode('utf-8'))

df2014['strRazaoSocial'] = df2014['strRazaoSocial'].apply(remove_acentuacao)

# Verificando a razão social de marcas famosas
SAMSUNG, NOKIA, MOTOROLA..

In [37]:
df2014[df2014['strRazaoSocial'].str.startswith('NOKIA ')]['strRazaoSocial'].unique()#samsung informática

array(['NOKIA DO BRASIL TECNOLOGIA LTDA',
       'NOKIA DO BRASIL TECNOLOGIA LTDA.', 'NOKIA DO BRASIL S.A',
       'NOKIA CELULARES', 'NOKIA DO BRASIL TECONOLOGIA LTDA',
       'NOKIA DO BRASIL TECNOLOGIA LTDA - AOP',
       'NOKIA DO BRASIL TECNOLOGIA', 'NOKIA STORE',
       'NOKIA CONECTING PEAPOLE'], dtype=object)

VIVO, TIM, OI, CLARO..

In [32]:
df2014[df2014['strRazaoSocial'].str.startswith('OI ')]['strRazaoSocial'].unique() # Nome Fantasia: OI

array(['OI MOVEL S.A.', 'OI MOVEL S/A', 'OI S.A', 'OI S.A.',
       'OI MOVEL S.A', 'OI S/A',
       'OI MOVEL S/A (14 BRASIL TELECOM CELULAR S/A)', 'OI S.A. (OI FIXO)',
       'OI SA', 'OI - TNL PCS S/A', 'OI MOVEL S/A.', 'OI MOVEL SA',
       'OI S/A KENNEDY NORONHA TERENCE', 'OI TV', 'OI MOVEL / OI TV',
       'OI MOVEL S/A (ANTIGA TNL)', 'OI S.A. BRASIL TELECOM',
       'OI SERVICOS DE TELECOMUNIACOES S.A.', 'OI S.A. (TELEFONIA FIXA)',
       'OI S/A.', 'OI S/A - SERVICO DE TELEFONIA FIXA COMUTADA',
       'OI MOVEL  S/A', 'OI - 14 BRASIL TELECOM CELULAR S/A',
       'OI MOVEL S.A. - (CNPJ COM SITUACAO CADASTRAL RECEI',
       'OI - TELEMAR NORTE LESTE S/A, OI S/A, OI MOVEL S/A',
       'OI TNL PCS S.A', 'OI VALE', 'OI S.A. (OI FIXO / BRASIL TELECOM)',
       'OI PAGGO ADMINISTRADORA DE CREDITO LTDA', 'OI CELULAR S.A',
       'OI MOVEL S/A - 14 BRASIL TELECOM CELULAR S/A',
       'OI - BRASIL TELECOM S/A'], dtype=object)

# Criando novo arquivo com todos os anos
Data de Abertura: 2013 a 2016

In [4]:
def summarise_data(filenames, output_file):
    with open(output_file, 'wb') as master_file:
        out_colnames = ['DataArquivamento', 'DataAbertura','Regiao', 'UF', 'RazaoSocial', 
                        'Atendida', 'Assunto','Problema', 'FaixaEtaria']   
        
        trip_writer = csv.DictWriter(master_file, fieldnames = out_colnames, delimiter=';')
        trip_writer.writeheader()
        
        for filename in filenames:
            with open(filename, 'rb') as f_in:
                # configura o leitor do csv
                trip_reader = csv.DictReader(f_in)
                
                # processa cada linha lendo uma a uma
                for row in trip_reader:
                    if datetime.strptime(row['DataAbertura'], '%Y-%m-%d %H:%M:%S.%f') >= datetime.strptime('2013-01-01 00:00:00.000', '%Y-%m-%d %H:%M:%S.%f'):
                        new_point = {}

                        new_point['DataArquivamento'] = datetime.strptime(row['DataArquivamento'], '%Y-%m-%d %H:%M:%S.%f')
                        new_point['DataAbertura'] = datetime.strptime(row['DataAbertura'], '%Y-%m-%d %H:%M:%S.%f')
                        new_point['Regiao'] = row['Regiao']
                        new_point['UF'] = row['UF']
                        
                        #corrigindo nomes com Sociedade Anônima
                        if 'S/A' in row['strRazaoSocial']:
                            row['strRazaoSocial'] = row['strRazaoSocial'].replace('S/A', 'SA')
                        if 'S.A' in row['strRazaoSocial']:
                            row['strRazaoSocial'] = row['strRazaoSocial'].replace('S.A', 'SA')
                        
                        
                        new_point['RazaoSocial'] = unidecode.unidecode(row['strRazaoSocial'].decode('utf-8'))
                        
                        #agrupando empresas/marcas conhecidas
                        if row['strRazaoSocial'].startswith('SAMSUNG ') or (
                        row['strRazaoSocial'].startswith('NOKIA ') ) or (
                        row['strRazaoSocial'].startswith('MOTOROLA ') ) or (
                        row['strRazaoSocial'].startswith('LG ') ) or (
                        row['strRazaoSocial'].startswith('SONY ') ) or (
                        row['strRazaoSocial'].startswith('DELL ') ) or (
                        row['strRazaoSocial'].startswith('LENOVO ') ) or (
                        row['strRazaoSocial'].startswith('ACER DO BRASIL ') ) or (
                        row['strRazaoSocial'].startswith('CEMAZ ') ):
                            split_name = row['strRazaoSocial'].split(" ")
                            new_point['RazaoSocial'] = split_name[0]
                            
                        #agrupando as empresas de telefonia
                        if row['strRazaoSocial'].startswith('TNL '):
                            new_point['RazaoSocial'] = 'OI'
                            
                        if row['strRazaoSocial'].startswith('CLARO ') or (
                        row['strRazaoSocial'].startswith('TIM ') ) or (
                        row['strRazaoSocial'].startswith('OI ') ) or (
                        row['strRazaoSocial'].startswith('VIVO ') ) or (
                        row['strRazaoSocial'].startswith('TELEMAR ') ):
                            split_name = row['strRazaoSocial'].split(" ")
                            new_point['RazaoSocial'] = split_name[0]
                            
                        new_point['Assunto'] = row['DescricaoAssunto']
                        new_point['Problema'] = row['DescricaoProblema']
                        new_point['FaixaEtaria'] = row['FaixaEtariaConsumidor']

                        if row['Atendida'] == 'S':
                            new_point['Atendida'] = 'Atendido'
                        else:
                            new_point['Atendida'] = 'Não Atendido'

                        trip_writer.writerow(new_point);

In [5]:
trip_in = ['reclamacoes-fundamentadas-sindec-2013.csv',
           'reclamacoes-fundamentadas-sindec-2014.csv',
           'reclamacoes-fundamentadas-sindec-2015.csv',
           'reclamacoes-fundamentadas-sindec-2016.csv'
          ]
trip_out = 'reclamacoes-fundamentadas-sindec-2013-2016.csv'

# Esta função irá ler as informações dos arquivos
# e escreverá um arquivo processado com o nome trip_out
summarise_data(trip_in, trip_out)

# Quantidade de dados
após gerar novo arquivo com a função summarise_data

In [6]:
df = pd.read_csv('reclamacoes-fundamentadas-sindec-2013-2016.csv', delimiter=';')
df['DataAbertura'].count()

856233

In [7]:
df.tail()

,DataArquivamento,DataAbertura,Regiao,UF,RazaoSocial,Atendida,Assunto,Problema,FaixaEtaria
856228,2016-08-24 10:19:29,2016-05-06 11:21:45,Sudeste,SP,OI,Atendida,Telefonia Celular,Contrato - Rescisão/alteração unilateral,entre 51 a 60 anos
856229,2016-10-27 09:19:09,2016-05-02 15:28:57,Sudeste,SP,ELETROPAULO METROPOLITANA ELETRICIDADE DE S PAULO,Não Atendida,Energia Elétrica,Cobrança indevida/abusiva,entre 41 a 50 anos
856230,2016-09-21 09:11:21,2016-04-29 16:20:30,Sudeste,SP,AUTO ESCOLA ULTRA FACIL LTDA EPP,Atendida,Automóvel - Locação / Assistência Automobilíst...,Serviço não concluído /Fornecimento parcial,entre 21 a 30 anos
856231,2016-08-16 10:58:52,2016-04-19 07:58:16,Sudeste,SP,FINANCEIRA ITAU CBD SA CREDITO FINANCIAMENTO E...,Atendida,Banco comercial,Cobrança indevida/abusiva,entre 21 a 30 anos
856232,2016-09-21 09:44:57,2016-04-19 08:42:27,Sudeste,SP,VALBET CENTRO AUTOMOTIVO LTDA ME,Atendida,"Oficinas ( Mecânica, Funilaria, Auto-Elétrica,...","Vicio de qualidade (mal executado, inadequado,...",entre 51 a 60 anos
